## TP parte A - Ruleta, Crossover Mutacion

### Librerias

In [18]:
import numpy as np
import funciones as f


### Variables

In [19]:
PROB_CROSSOVER = 0.75
PROB_MUTACION = 0.05
CICLOS = 200
CANTIDAD_POBLACION = 10
COEF = 1073741823 # 2^29

### Poblacion

#### Decimales

In [20]:
rng = np.random.default_rng() # Random Number Generator
poblacion_decimal = rng.integers(low=0, high=COEF, size=CANTIDAD_POBLACION)
poblacion_decimal

array([ 321542348,  514938062,  720918912, 1015637325,  303354065,
       1024793637,  322300738,  425859065,  853106008,  370838857],
      dtype=int64)

#### Binarios

In [21]:
poblacion_binarios = f.convertirPoblacionBin(poblacion_decimal)
poblacion_binarios

['010011001010100101100011001100',
 '011110101100010101010011001110',
 '101010111110000101100110000000',
 '111100100010010110010101001101',
 '010010000101001101000011010001',
 '111101000101010001110000100101',
 '010011001101011110101101000010',
 '011001011000100001011111111001',
 '110010110110010101110101011000',
 '010110000110101000110101001001']

### Tabla de datos

In [22]:
marcoDeDatos = f.generarDataFrame(poblacion_decimal, poblacion_binarios, COEF)
marcoDeDatos

,Decimales,Binarios,FuncionObjetivo,Fitness
0,321542348,010011001010100101100011001100,0.089676,0.024543
1,514938062,011110101100010101010011001110,0.229991,0.062945
2,720918912,101010111110000101100110000000,0.450789,0.123375
3,1015637325,111100100010010110010101001101,0.894700,0.244867
4,303354065,010010000101001101000011010001,0.079818,0.021845
5,1024793637,111101000101010001110000100101,0.910905,0.249302
6,322300738,010011001101011110101101000010,0.090100,0.024659
7,425859065,011001011000100001011111111001,0.157301,0.043051
8,853106008,110010110110010101110101011000,0.631257,0.172766
9,370838857,010110000110101000110101001001,0.119281,0.032646


#### Estadisticas Marco de datos

In [23]:
marcoDeDatos_stats = f.generarEstadisticas(marcoDeDatos)
marcoDeDatos_stats

,Suma,Promedio,Maximo
Función objetivo,3.653817,0.365382,0.910905
Fitness,1.000000,0.100000,0.249302


### Ruleta

In [24]:
cromosomasPadresRuleta = f.ruleta(marcoDeDatos,CANTIDAD_POBLACION)
cromosomasPadresRuleta

array(['111100100010010110010101001101', '111101000101010001110000100101',
       '111101000101010001110000100101', '111100100010010110010101001101',
       '111100100010010110010101001101', '010010000101001101000011010001',
       '111100100010010110010101001101', '101010111110000101100110000000',
       '011001011000100001011111111001', '111101000101010001110000100101'],
      dtype=object)

### Crossover

In [25]:
crossover = f.crossover(cromosomasPadresRuleta,PROB_CROSSOVER)
crossover

['111101000101010001110000100101',
 '111100100010010110010101001101',
 '111101000101010110010101001101',
 '111100100010010001110000100101',
 '111100100101001101000011010001',
 '010010000010010110010101001101',
 '111100101110000101100110000000',
 '101010110010010110010101001101',
 '011001011000100001011111111001',
 '111101000101010001110000100101']

### Mutacion

In [26]:
mutantes = f.mutacion(crossover, PROB_MUTACION)
mutantes

['111101000101010001110000100101',
 '111100100010010110010101001101',
 '111101000101010110010101001101',
 '111100100010010001110000100101',
 '111100100101001101000011010001',
 '010010000010010110010101001101',
 '111100101110000101100110000000',
 '101010110010010110010101001101',
 '011001011000100001011111111001',
 '111101000101010001110000100101']

### Tabla de datos Mutados

In [27]:
marcoDeDatos = f.generarDataFrame(f.convertirPoblacionDecimal(mutantes), mutantes, COEF)
marcoDeDatos

,Decimales,Binarios,FuncionObjetivo,Fitness
0,1024793637,111101000101010001110000100101,0.910905,0.130093
1,1015637325,111100100010010110010101001101,0.894700,0.127779
2,1024812365,111101000101010110010101001101,0.910938,0.130098
3,1015618597,111100100010010001110000100101,0.894667,0.127774
4,1016385745,111100100101001101000011010001,0.896019,0.127967
5,302605645,010010000010010110010101001101,0.079424,0.011343
6,1018714496,111100101110000101100110000000,0.900130,0.128554
7,717841741,101010110010010110010101001101,0.446949,0.063832
8,425859065,011001011000100001011111111001,0.157301,0.022465
9,1024793637,111101000101010001110000100101,0.910905,0.130093


In [28]:
marcoDeDatos_stats = f.generarEstadisticas(marcoDeDatos)
marcoDeDatos_stats

,Suma,Promedio,Maximo
Función objetivo,7.00194,0.700194,0.910938
Fitness,1.00000,0.100000,0.130098


## TP Parte B - Torneo, Crossover Mutacion

## TP Parte C - Ruleta, Elitismo, Crosover Mutacion